# Manejo de Datos en Google Colaboratory

## Moverse entre directorios


Para movernos entre directorios debemos utilizar 

```
import os 
os.chdir("path_to_file")
```

No podemos utilizar "cd" para cambiarnos de directorio

```
!cd path_to_file # Si hacemos esto no nos cambiaremos de directorio
```






## Cómo Cargar Datos externos en Colaboratory

Los datos pueden ser cargados de distintas formas en colaboratory, podemos cargarlos desde algún repositorio, descargarlos desde alguna página web o desde nuestros Drive.

Para clonar un repositorio utilizamos:

```
!git clone url
```

Para descargar directamente un archivo desde una página web utilizamos:



```
!wget url
```




### Ejemplo: Discursos de Piñera

In [1]:
!ls

datalab


In [0]:
# Remover archivo/carpeta si ya existe

import os

if os.path.exists('/content/piñera/'):
  !rm -R /content/piñera/


### !wget

In [3]:
!wget http://dcc.uchile.cl/~jperez/resources/piñera_1.tar.gz -P /content/piñera

--2018-03-19 02:09:05--  http://dcc.uchile.cl/~jperez/resources/pi%C3%B1era_1.tar.gz
Resolving dcc.uchile.cl (dcc.uchile.cl)... 192.80.24.4
Connecting to dcc.uchile.cl (dcc.uchile.cl)|192.80.24.4|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.dcc.uchile.cl/~jperez/resources/pi%c3%b1era_1.tar.gz [following]
--2018-03-19 02:09:05--  https://www.dcc.uchile.cl/~jperez/resources/pi%c3%b1era_1.tar.gz
Resolving www.dcc.uchile.cl (www.dcc.uchile.cl)... 192.80.24.4
Connecting to www.dcc.uchile.cl (www.dcc.uchile.cl)|192.80.24.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://users.dcc.uchile.cl/~jperez/resources/pi%c3%b1era_1.tar.gz [following]
--2018-03-19 02:09:06--  https://users.dcc.uchile.cl/~jperez/resources/pi%c3%b1era_1.tar.gz
Resolving users.dcc.uchile.cl (users.dcc.uchile.cl)... 192.80.24.4
Connecting to users.dcc.uchile.cl (users.dcc.uchile.cl)|192.80.24.4|:443... connected.
HTTP request sen

piñera_1.tar.gz     100%[===================>]   5.32M   113KB/s    in 38s     

2018-03-19 02:09:45 (143 KB/s) - ‘/content/piñera/piñera_1.tar.gz’ saved [5577286/5577286]



In [4]:
!ls

datalab  piñera


In [5]:
!ls piñera

piñera_1.tar.gz


## Cargar datos desde Google Drive

### Para cargar datos desde Drive debemos instalar PyDrive y otorgarle permisos a Colaboratory

In [0]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

## Nos debemos autentificar como usuarios
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

### Con esto podemos crear una carpeta localmente desde la cual leeremos y/o escribiremos archivos en Drive

In [0]:
local_download_path = os.path.expanduser('~/data_drive')
try:
  os.makedirs(local_download_path)
except: pass

### Para ello necesitamos la ID de nuestra carpeta en drive, esta se encuentra en la url de Drive cuando nos encontramos en la carpeta.
<img src="https://raw.githubusercontent.com/jorgeperezrojas/cc6204-DeepLearning-DCCUChile/master/2018/Auxiliares/Auxiliar1/img/id_drive_folder.png" width="835" height="45"/>


In [0]:
folder_id = "1ua5NDMXarsJk2R6SB937p95m0q9EDxDP"

### Con esto podemos descargar los archivos desde ese directorio en Drive

In [9]:
# Descarga de archivos !
file_list = drive.ListFile(
    {'q': "'{}' in parents".format(folder_id)}).GetList()

for f in file_list:
  # Creamos los archivos y los descargamos
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)

title: ./test1.txt, id: 1Omiukf0tvKeZdm-_kF5kjVdsvY-R6EmX
downloading to /content/data_drive/./test1.txt
title: ./test1.txt, id: 1aJc1Mt-K09s92hwQtclvRVc1eq7O-fP1
downloading to /content/data_drive/./test1.txt
title: bachelet_2.txt, id: 1L38Z0_q86wjOGTcWqaAI05dfSVxl-lwp
downloading to /content/data_drive/bachelet_2.txt


### Nos cambiaremos de directorio para poder crear archivos que luego subiremos a Drive

In [10]:
!ls

data_drive  datalab  piñera


In [11]:
import os
os.chdir("./data_drive")
!ls

bachelet_2.txt	test1.txt


### Para cargar archivos podemos utilizar el metodo CreateFile creando el archivo con Drive para luego subirlo a Drive

In [12]:
## Creamos un archivo con drive y la subimos a drive

path_test_file = "./test1.txt"
uploaded = drive.CreateFile({'title': path_test_file, "parents": [{"kind": "drive#fileLink","id": folder_id}]})


with open(path_test_file, "w") as f1:
  f1.write("Archivo de prueba\n")
  
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1K-Ex2YSfBbNHjEdkuEA5dIf2WhJ91fhf


### O utilizar MediaFileUpload para subir archivos ya creados a Drive.

In [13]:
## Creamos un archivo con python y lo subimos a Drive

from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

path_test_file = "./test2.txt"

with open(path_test_file, "w") as f1:
  f1.write("Archivo de prueba 2\n")
  
  
file_metadata = {
  'name': 'test2',
  'mimeType': 'text/plain',
   "parents": [{"kind": "drive#fileLink","id": folder_id}]
}

media = MediaFileUpload(path_test_file, 
                        mimetype='text/plain',
                        resumable=True)

created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()

print('File ID: {}'.format(created.get('id')))

File ID: 1tbv81YAMS_lsp9ts8geYAxdZYfWjc5MU
